In [28]:
# Import packages

import pandas as pd 
import numpy as np 
import os
from bs4 import BeautifulSoup
import time
import requests
import nfl_data_py as nfl
import matplotlib.pyplot as plt
import plotly
import plotly.express as px

# Set certain settings for the notebook
pd.set_option("display.max_columns", None)

In [29]:
# Check current directory
os.getcwd()

'C:\\Users\\imacd_0odruq3\\Documents\\sports_gambling\\sports_gambling_algorithm'

In [30]:
# UPDATE THIS before each run

current_week = 12
current_season = 2022

In [31]:
def fix_team_names(team):
    team_mapping = {
        'ARI':'Arizona Cardinals',
        'ATL':'Atlanta Falcons',
        'BAL':'Baltimore Ravens',
        'BUF':'Buffalo Bills',
        'CAR':'Carolina Panthers',
        'CHI':'Chicago Bears',
        'CIN':'Cincinnati Bengals',
        'CLE':'Cleveland Browns',
        'DAL':'Dallas Cowboys',
        'DEN':'Denver Broncos',
        'DET':'Detroit Lions',
        'GB':'Green Bay Packers',
        'HOU':'Houston Texans',
        'IND':'Indianapolis Colts',
        'JAX':'Jacksonville Jaguars',
        'KC':'Kansas City Chiefs',
        'OAK':'Las Vegas Raiders',
        'LV':'Las Vegas Raiders',
        'LAC':'Los Angeles Chargers',
        'LAR':'Los Angeles Rams',
        'MIA':'Miami Dolphins',
        'MIN':'Minnesota Vikings',
        'NE':'New England Patriots',
        'NO':'New Orleans Saints',
        'NYG':'New York Giants',
        'NYJ':'New York Jets',
        'PHI':'Philadelphia Eagles',
        'PIT':'Pittsburgh Steelers',
        'SF':'San Francisco 49ers',
        'SEA':'Seattle Seahawks',
        'TB':'Tampa Bay Buccaneers',
        'TEN':'Tennessee Titans',
        'WAS':'Washington Football Team',
        'WSH':'Washington Football Team'
    }
    
    return team_mapping[team]

In [32]:
# Function to get quarterbacks
# Uses last available week as the current qb 
# IMPORTANT NOTE: QBs change teams during the offseason, or are traded during the season, get injured, etc. 
# CHECK to make sure the correct qb is noted for each team. 

elo_qb_data = pd.read_csv('../data/elo_qb_data.csv')

if current_week == 1:
    reference_season = current_season-1
    reference_week = 17
    
else:
    reference_season = current_season
    reference_week = current_week - 1
    
elo_qb_data = elo_qb_data[(elo_qb_data.season == reference_season) & (elo_qb_data.week == reference_week)]
# elo_qb_data

In [33]:
qbs = {game['home_full_name']:game['qb1'] for index, game in elo_qb_data.iterrows()}
away_qbs = {game['away_full_name']:game['qb2'] for index, game in elo_qb_data.iterrows()}
qbs.update(away_qbs)

In [34]:
# QB Updates, if necessary

qbs['Arizona Cardinals'] = 'Kyler Murray'
qbs['Atlanta Falcons'] = 'Marcus Mariota'
qbs['Baltimore Ravens'] = 'Lamar Jackson'
qbs['Buffalo Bills'] = 'Josh Allen'
qbs['Carolina Panthers'] = 'P.J. Walker'
qbs['Chicago Bears'] = 'Trevor Siemian'
qbs['Cincinnati Bengals'] = 'Joe Burrow'
qbs['Cleveland Browns'] = 'Jacoby Brissett'
qbs['Dallas Cowboys'] = 'Dak Prescott'
qbs['Denver Broncos'] = 'Russell Wilson'
qbs['Detroit Lions'] = 'Jared Goff'
qbs['Green Bay Packers'] = 'Aaron Rodgers'
qbs['Houston Texans'] = 'Kyle Allen'
qbs['Indianapolis Colts'] = 'Matt Ryan'
qbs['Jacksonville Jaguars'] = 'Trevor Lawrence'
qbs['Kansas City Chiefs'] = 'Patrick Mahomes'
qbs['Las Vegas Raiders'] = 'Derek Carr'
qbs['Los Angeles Chargers'] = 'Justin Herbert'
qbs['Los Angeles Rams'] = 'Matthew Stafford'
qbs['Miami Dolphins'] = 'Tua Tagovailoa'
qbs['Minnesota Vikings'] = 'Kirk Cousins'
qbs['New England Patriots'] = 'Mac Jones'
qbs['New Orleans Saints'] = 'Andy Dalton'
qbs['New York Giants'] = 'Daniel Jones'
qbs['New York Jets'] = 'Mike White'
qbs['Philadelphia Eagles'] = 'Jalen Hurts'
qbs['Pittsburgh Steelers'] = 'Kenny Pickett'
qbs['San Francisco 49ers'] = 'Jimmy Garoppolo'
qbs['Seattle Seahawks'] = 'Geno Smith'
qbs['Tampa Bay Buccaneers'] = 'Tom Brady'
qbs['Tennessee Titans'] = 'Ryan Tannehill'
qbs['Washington Football Team'] = 'Taylor Heinicke'

In [35]:
# CHECK: Are qbs correct?

# IF NOT, update in previous step. Otherwise, good to go

for team, qb in qbs.items():
    print('{}: {}'.format(team, qb))
    print()

Green Bay Packers: Aaron Rodgers

Houston Texans: Kyle Allen

Indianapolis Colts: Matt Ryan

New York Giants: Daniel Jones

Baltimore Ravens: Lamar Jackson

Atlanta Falcons: Marcus Mariota

Buffalo Bills: Josh Allen

New England Patriots: Mac Jones

New Orleans Saints: Andy Dalton

Denver Broncos: Russell Wilson

Minnesota Vikings: Kirk Cousins

Pittsburgh Steelers: Kenny Pickett

Los Angeles Chargers: Justin Herbert

Arizona Cardinals: Kyler Murray

Tennessee Titans: Ryan Tannehill

Washington Football Team: Taylor Heinicke

Philadelphia Eagles: Jalen Hurts

Detroit Lions: Jared Goff

Carolina Panthers: P.J. Walker

Chicago Bears: Trevor Siemian

Cleveland Browns: Jacoby Brissett

New York Jets: Mike White

Los Angeles Rams: Matthew Stafford

Las Vegas Raiders: Derek Carr

Dallas Cowboys: Dak Prescott

Cincinnati Bengals: Joe Burrow

Kansas City Chiefs: Patrick Mahomes

San Francisco 49ers: Jimmy Garoppolo

Jacksonville Jaguars: Trevor Lawrence

Miami Dolphins: Tua Tagovailoa

Seattle

In [36]:
# Manually Inputted
# While not ideal, manually inputting the odds/games in doesn't take too much time/effort
# In the future, want to create a web scraping bot to automate getting the necessary odds and game data
# For now, this will have to do

# Column Order
# 'season', 'week', 'away', 'home', 'away_moneyline', 'home_moneyline', 'away_spread', 'home_spread', 'over_under'

manual_input = {
    '{}{}{}'.format(current_season, current_week, 1): [current_season, current_week, 'BUF','DET',
                                                      -410, 330, -9.5, 9.5, 54.5],
    '{}{}{}'.format(current_season, current_week, 2): [current_season, current_week, 'NYG','DAL',
                                                      350, -435, 10, -10, 45.5],
    '{}{}{}'.format(current_season, current_week, 3): [current_season, current_week, 'NE','MIN',
                                                      115, -135, 2.5, -2.5, 42.5],
    '{}{}{}'.format(current_season, current_week, 4): [current_season, current_week, 'HOU','MIA',
                                                      550, -750, 13.5, -13.5, 47],
    '{}{}{}'.format(current_season, current_week, 5): [current_season, current_week, 'BAL','JAX',
                                                      -195, 165, -3.5, 3.5, 43.5],
    '{}{}{}'.format(current_season, current_week, 6): [current_season, current_week, 'CHI','NYJ',
                                                      215, -255, 6, -6, 38.5],
    '{}{}{}'.format(current_season, current_week, 7): [current_season, current_week, 'CIN','TEN',
                                                      -150, 130, -2.5, 2.5, 42.5],
    '{}{}{}'.format(current_season, current_week, 8): [current_season, current_week, 'ATL','WAS',
                                                      170, -200, 4.5, -4.5, 40.5],
    '{}{}{}'.format(current_season, current_week, 9): [current_season, current_week, 'DEN','CAR',
                                                      -130, 110, -1.5, 1.5, 36],
    '{}{}{}'.format(current_season, current_week, 10): [current_season, current_week, 'TB','CLE',
                                                      -180, 155, -3.5, 3.5, 42],
    '{}{}{}'.format(current_season, current_week, 11): [current_season, current_week, 'LV','SEA',
                                                      165, -195, 3.5, -3.5, 48],
    '{}{}{}'.format(current_season, current_week, 12): [current_season, current_week, 'LAC','ARI',
                                                      -175, 150, -3, 3, 48],
    '{}{}{}'.format(current_season, current_week, 13): [current_season, current_week, 'LAR','KC',
                                                      750, -1150, 15.5, -15.5, 42],
    '{}{}{}'.format(current_season, current_week, 14): [current_season, current_week, 'NO','SF',
                                                      330, -410, 9.5, -9.5, 43.5],
    '{}{}{}'.format(current_season, current_week, 15): [current_season, current_week, 'GB','PHI',
                                                      245, -295, 6.5, -6.5, 43.5]
#     '{}{}{}'.format(current_season, current_week, 16): [current_season, current_week, 'LV','KC',
#                                                       265, -320, 7, -7, 51.5]
}

In [37]:
manual_input

{'2022121': [2022, 12, 'BUF', 'DET', -410, 330, -9.5, 9.5, 54.5],
 '2022122': [2022, 12, 'NYG', 'DAL', 350, -435, 10, -10, 45.5],
 '2022123': [2022, 12, 'NE', 'MIN', 115, -135, 2.5, -2.5, 42.5],
 '2022124': [2022, 12, 'HOU', 'MIA', 550, -750, 13.5, -13.5, 47],
 '2022125': [2022, 12, 'BAL', 'JAX', -195, 165, -3.5, 3.5, 43.5],
 '2022126': [2022, 12, 'CHI', 'NYJ', 215, -255, 6, -6, 38.5],
 '2022127': [2022, 12, 'CIN', 'TEN', -150, 130, -2.5, 2.5, 42.5],
 '2022128': [2022, 12, 'ATL', 'WAS', 170, -200, 4.5, -4.5, 40.5],
 '2022129': [2022, 12, 'DEN', 'CAR', -130, 110, -1.5, 1.5, 36],
 '20221210': [2022, 12, 'TB', 'CLE', -180, 155, -3.5, 3.5, 42],
 '20221211': [2022, 12, 'LV', 'SEA', 165, -195, 3.5, -3.5, 48],
 '20221212': [2022, 12, 'LAC', 'ARI', -175, 150, -3, 3, 48],
 '20221213': [2022, 12, 'LAR', 'KC', 750, -1150, 15.5, -15.5, 42],
 '20221214': [2022, 12, 'NO', 'SF', 330, -410, 9.5, -9.5, 43.5],
 '20221215': [2022, 12, 'GB', 'PHI', 245, -295, 6.5, -6.5, 43.5]}

In [38]:
current_week_data = pd.DataFrame.from_dict(manual_input, orient='index',
                                          columns = ['season', 'week', 'away', 'home', 
                                                    'away_moneyline', 'home_moneyline', 
                                                    'away_spread', 'home_spread', 'over_under'])
current_week_data['home_full_name'] = current_week_data.apply(lambda x: fix_team_names(x.home), axis=1)
current_week_data['away_full_name'] = current_week_data.apply(lambda x: fix_team_names(x.away), axis=1)
current_week_data['home_qb'] = current_week_data.apply(lambda x: qbs[x.home_full_name], axis=1)
current_week_data['away_qb'] = current_week_data.apply(lambda x: qbs[x.away_full_name], axis=1)

In [39]:
current_week_data

,season,week,away,home,away_moneyline,home_moneyline,away_spread,home_spread,over_under,home_full_name,away_full_name,home_qb,away_qb
2022121,2022,12,BUF,DET,-410,330,-9.5,9.5,54.5,Detroit Lions,Buffalo Bills,Jared Goff,Josh Allen
2022122,2022,12,NYG,DAL,350,-435,10.0,-10.0,45.5,Dallas Cowboys,New York Giants,Dak Prescott,Daniel Jones
2022123,2022,12,NE,MIN,115,-135,2.5,-2.5,42.5,Minnesota Vikings,New England Patriots,Kirk Cousins,Mac Jones
2022124,2022,12,HOU,MIA,550,-750,13.5,-13.5,47.0,Miami Dolphins,Houston Texans,Tua Tagovailoa,Kyle Allen
2022125,2022,12,BAL,JAX,-195,165,-3.5,3.5,43.5,Jacksonville Jaguars,Baltimore Ravens,Trevor Lawrence,Lamar Jackson
2022126,2022,12,CHI,NYJ,215,-255,6.0,-6.0,38.5,New York Jets,Chicago Bears,Mike White,Trevor Siemian
2022127,2022,12,CIN,TEN,-150,130,-2.5,2.5,42.5,Tennessee Titans,Cincinnati Bengals,Ryan Tannehill,Joe Burrow
2022128,2022,12,ATL,WAS,170,-200,4.5,-4.5,40.5,Washington Football Team,Atlanta Falcons,Taylor Heinicke,Marcus Mariota
2022129,2022,12,DEN,CAR,-130,110,-1.5,1.5,36.0,Carolina Panthers,Denver Broncos,P.J. Walker,Russell Wilson
20221210,2022,12,TB,CLE,-180,155,-3.5,3.5,42.0,Cleveland Browns,Tampa Bay Buccaneers,Jacoby Brissett,Tom Brady


In [40]:
current_week_data.to_csv('../data/current_week_data.csv')